In [1]:
from dataset.build_dataset import build_dataset
import mindspore.dataset as ds
import os
import numpy as np
from mindspore.mindrecord import FileWriter
import json

[WARNING] ME(12621:140269430196032,MainProcess):2021-10-11-12:20:10.663.990 [mindspore/run_check/_check_version.py:181] Cuda ['10.1', '11.1'] version(need by mindspore-gpu) is not found, please confirm that the path of cuda is set to the env LD_LIBRARY_PATH, please refer to the installation guidelines: https://www.mindspore.cn/install
[WARNING] ME(12621:140269430196032,MainProcess):2021-10-11-12:20:10.671.110 [mindspore/run_check/_check_version.py:181] Cuda ['10.1', '11.1'] version(need by mindspore-gpu) is not found, please confirm that the path of cuda is set to the env LD_LIBRARY_PATH, please refer to the installation guidelines: https://www.mindspore.cn/install


In [2]:
FEATURES_FILE = "./data/train_data.npy"

features = np.load(FEATURES_FILE)
list_dict = []
for item in features:
    dict_temp = json.loads(item)
    list_dict.append(dict_temp)

In [3]:
len(list_dict[0]['word_ids'])

180

In [4]:
list_dict[0]['word_ids']

[0,
 598,
 2661,
 222,
 5,
 9880,
 2708,
 2346,
 2082,
 11,
 504,
 4432,
 11,
 226,
 2126,
 10067,
 1470,
 116,
 2,
 2,
 29474,
 28108,
 6,
 5,
 334,
 34,
 10,
 4019,
 2048,
 4,
 497,
 1517,
 5,
 4326,
 6919,
 18,
 1637,
 31346,
 16,
 10,
 9030,
 9577,
 9,
 5,
 9880,
 2708,
 4,
 29261,
 11,
 760,
 9,
 5,
 4326,
 6919,
 8,
 2114,
 24,
 6,
 16,
 10,
 7621,
 9577,
 9,
 4845,
 19,
 3701,
 62,
 33161,
 19,
 5,
 7875,
 22,
 39043,
 1459,
 1614,
 1464,
 13292,
 4977,
 845,
 4130,
 7,
 5,
 4326,
 6919,
 16,
 5,
 26429,
 2426,
 9,
 5,
 25095,
 6924,
 4,
 29261,
 639,
 5,
 32394,
 2426,
 16,
 5,
 7461,
 26187,
 6,
 10,
 19035,
 317,
 9,
 9621,
 8,
 12456,
 4,
 85,
 16,
 10,
 24633,
 9,
 5,
 11491,
 26187,
 23,
 226,
 2126,
 10067,
 6,
 1470,
 147,
 5,
 9880,
 2708,
 2851,
 13735,
 352,
 1382,
 7,
 6130,
 6552,
 625,
 3398,
 208,
 22895,
 853,
 1827,
 11,
 504,
 4432,
 4,
 497,
 5,
 253,
 9,
 5,
 1049,
 1305,
 36,
 463,
 11,
 10,
 2228,
 516,
 14,
 15230,
 149,
 155,
 19638,
 8,
 5,
 2610,
 25336

In [5]:
SQUAD_MINDRECORD_FILE = "./data/train_features.mindrecord"

pad = lambda a,i : a[0:i] if len(a) > i else a + [0] * (i-len(a))
pad1 = lambda a,i : a[0:i] if len(a) > i else a + [1] * (i-len(a))
pad_entity = lambda a,i : a[0:i] if len(a) > i else np.append(a,[-1] * (i-len(a)))

for slist in list_dict:
    slist["entity_attention_mask"] = pad(slist["entity_attention_mask"], 24)
    slist["entity_ids"] = pad(slist["entity_attention_mask"], 24)
    slist["entity_segment_ids"] = pad(slist["entity_segment_ids"], 24)
    
    slist["word_ids"] = pad1(slist["word_ids"], 256)
    slist["word_segment_ids"] = pad(slist["word_segment_ids"], 256)
    slist["word_attention_mask"] = pad(slist["word_attention_mask"], 256)
    slist["entity_position_ids"] = np.array(slist["entity_position_ids"]).flatten()
    slist["entity_position_ids"] = pad_entity(slist["entity_position_ids"], 256)


if os.path.exists(SQUAD_MINDRECORD_FILE):
    os.remove(SQUAD_MINDRECORD_FILE)
    os.remove(SQUAD_MINDRECORD_FILE + ".db")

writer = FileWriter(file_name=SQUAD_MINDRECORD_FILE, shard_num=1)

data_schema = {
    #"unique_id": {"type": "int32", "shape": [-1]},
    "word_ids": {"type": "int32", "shape": [-1]},
    "word_segment_ids": {"type": "int32", "shape": [-1]},
    "word_attention_mask": {"type": "int32", "shape": [-1]},
    "entity_ids": {"type": "int32", "shape": [-1]},
    "entity_position_ids": {"type": "int32", "shape": [-1]},
    "entity_segment_ids": {"type": "int32", "shape": [-1]},
    "entity_attention_mask": {"type": "int32", "shape": [-1]},
    "start_positions": {"type": "int32", "shape": [-1]},
    "end_positions": {"type": "int32", "shape": [-1]}
}
writer.add_schema(data_schema, "it is a preprocessed squad dataset")

data = []
i = 0
for item in list_dict:
    i += 1
    sample = {
        #"unique_id": np.array(item["unique_id"], dtype=np.int32),
        "word_ids": np.array(item["word_ids"], dtype=np.int32),
        "word_segment_ids": np.array(item["word_segment_ids"], dtype=np.int32),
        "word_attention_mask": np.array(item["word_attention_mask"], dtype=np.int32),
        "entity_ids": np.array(item["entity_ids"], dtype=np.int32),
        "entity_position_ids": np.array(item["entity_position_ids"], dtype=np.int32),
        "entity_segment_ids": np.array(item["entity_segment_ids"], dtype=np.int32),
        "entity_attention_mask": np.array(item["entity_attention_mask"], dtype=np.int32),
        "start_positions": np.array(item["start_positions"], dtype=np.int32),
        "end_positions": np.array(item["end_positions"], dtype=np.int32),
    }

    data.append(sample)
    #print(sample)
    if i % 10 == 0:
        writer.write_raw_data(data)
        data = []
print(data[0])
if data:
    writer.write_raw_data(data)

writer.commit()

{'word_ids': array([    0,  1740,    99,   343,   473,  1586,  1071, 19252,   904,
         786,  8287,  4871,     7, 15359, 14785,   257,   116,     2,
           2,    20,  1049,   758,  3062,  2754, 15359, 14785,   257,
           8,  4634, 15377,    16,     5, 24311,  6455,  9965,  1016,
        4414,     6,  2034,    59,   411,  8130,    36,   401,  6301,
          36,   246,     4,   406, 11163, 35122,    31,     5,   343,
        2100,     4, 23701,  1070,    30,     5,  5280, 10333,  4305,
           9, 15377,    24,    34,    80, 20531,     6,    65,  1897,
           8,    65,   758,     4,   497,  1455,     6,    59,   820,
         758,  8537,  4686, 15377,     7,    97, 11633,    11,  1005,
           6,  1817,     8,     5,  2367,   953,     6,     7,  1947,
         215,    25, 12275,     6,  3534,     6,   229, 14024,  2186,
           6,  2920,     6, 15398,     6, 18959, 19193,     6,  8086,
        5870,     6, 12600,     6,  2884,   139,     6,   226,  7333,
       

MSRStatus.SUCCESS

In [11]:
columns_list = ['word_ids',  'word_segment_ids', 'word_attention_mask',
               'entity_ids', 'entity_position_ids', 'entity_segment_ids',
               'entity_attention_mask', 'start_positions', 'end_positions']
output_features = np.array(list_dict)
#SQUAD_MINDRECORD_FILE = "./data/dev_features.mindrecord"
data_set = ds.MindDataset(dataset_file=SQUAD_MINDRECORD_FILE, columns_list=columns_list)
count = 0
for item in data_set.create_dict_iterator():
    #print(item)
    count += 1
print("Got {} samples".format(count))

Got 87777 samples


In [12]:
data_set = data_set.batch(1)# , drop_remainder=True
data_sample = next(data_set.create_dict_iterator())
data_sample

{'word_ids': Tensor(shape=[1, 256], dtype=Int32, value=
 [[   0, 1336,  171 ...    1,    1,    1]]),
 'word_segment_ids': Tensor(shape=[1, 256], dtype=Int32, value=
 [[0, 0, 0 ... 0, 0, 0]]),
 'word_attention_mask': Tensor(shape=[1, 256], dtype=Int32, value=
 [[1, 1, 1 ... 0, 0, 0]]),
 'entity_ids': Tensor(shape=[1, 24], dtype=Int32, value=
 [[1, 1, 1 ... 0, 0, 0]]),
 'entity_position_ids': Tensor(shape=[1, 256], dtype=Int32, value=
 [[12, 13, 14 ... -1, -1, -1]]),
 'entity_segment_ids': Tensor(shape=[1, 24], dtype=Int32, value=
 [[0, 0, 0 ... 0, 0, 0]]),
 'entity_attention_mask': Tensor(shape=[1, 24], dtype=Int32, value=
 [[1, 1, 1 ... 0, 0, 0]]),
 'start_positions': Tensor(shape=[1, 1], dtype=Int32, value=
 [[96]]),
 'end_positions': Tensor(shape=[1, 1], dtype=Int32, value=
 [[96]])}

In [13]:
data_sample['entity_attention_mask']

Tensor(shape=[1, 24], dtype=Int32, value=
[[1, 1, 1 ... 0, 0, 0]])

## Train

In [14]:
from readingcomprehension.models.luke import LukeForReadingComprehension, LukeForReadingComprehensionWithLoss, LukeEntityAwareAttentionModel, LukeSquadCell
import mindspore.common.dtype as mstype
from model.bert_model import BertConfig
from mindspore import context
from model.luke import LukeModel, EntityAwareEncoder
from mindspore import Tensor, context
from mindspore import dtype as mstype
import mindspore.ops as ops
import mindspore.nn as nn
from model.bert_model import BertOutput
from mindspore.common.initializer import TruncatedNormal
from mindspore.ops import composite as C
import mindspore
from mindspore.ops import operations as P
from mindspore.train.model import Model
from tqdm import tqdm
from mindspore.train.serialization import load_checkpoint, load_param_into_net
from mindspore.train.model import Model
import collections
# PYNATIVE_MODE & GRAPH_MODE
context.set_context(mode=context.PYNATIVE_MODE, device_target="GPU")
context.set_context(enable_graph_kernel=True)

In [15]:

# def do_train(dataset=None, network=None, load_checkpoint_path="", save_checkpoint_path="", epoch_num=1):
#     """ do train """
#     if load_checkpoint_path == "":
#         raise ValueError("Pretrain model missed, finetune task must load pretrain model!")
#     steps_per_epoch = dataset.get_dataset_size()
#     # optimizer
#     if optimizer_cfg.optimizer == 'AdamWeightDecay':
#         lr_schedule = BertLearningRate(learning_rate=15e-6,
#                                        end_learning_rate=optimizer_cfg.AdamWeightDecay.end_learning_rate,
#                                        warmup_steps=int(steps_per_epoch * epoch_num * 0.1),
#                                        decay_steps=steps_per_epoch * epoch_num,
#                                        power=optimizer_cfg.AdamWeightDecay.power)
#         params = network.trainable_params()
#         decay_params = list(filter(optimizer_cfg.AdamWeightDecay.decay_filter, params))
#         other_params = list(filter(lambda x: not optimizer_cfg.AdamWeightDecay.decay_filter(x), params))
#         group_params = [{'params': decay_params, 'weight_decay': optimizer_cfg.AdamWeightDecay.weight_decay},
#                         {'params': other_params, 'weight_decay': 0.0}]

#         optimizer = AdamWeightDecay(group_params, lr_schedule, eps=optimizer_cfg.AdamWeightDecay.eps)
#     elif optimizer_cfg.optimizer == 'Lamb':
#         lr_schedule = BertLearningRate(learning_rate=optimizer_cfg.Lamb.learning_rate,
#                                        end_learning_rate=optimizer_cfg.Lamb.end_learning_rate,
#                                        warmup_steps=int(steps_per_epoch * epoch_num * 0.1),
#                                        decay_steps=steps_per_epoch * epoch_num,
#                                        power=optimizer_cfg.Lamb.power)
#         optimizer = Lamb(network.trainable_params(), learning_rate=lr_schedule)
#     elif optimizer_cfg.optimizer == 'Momentum':
#         optimizer = Momentum(network.trainable_params(), learning_rate=optimizer_cfg.Momentum.learning_rate,
#                              momentum=optimizer_cfg.Momentum.momentum)
#     else:
#         raise Exception("Optimizer not supported. support: [AdamWeightDecay, Lamb, Momentum]")

#     # load checkpoint into network
#     ckpt_config = CheckpointConfig(save_checkpoint_steps=steps_per_epoch, keep_checkpoint_max=1)
#     ckpoint_cb = ModelCheckpoint(prefix="squad",
#                                  directory=None if save_checkpoint_path == "" else save_checkpoint_path,
#                                  config=ckpt_config)
#     param_dict = load_checkpoint(load_checkpoint_path)
#     load_param_into_net(network, param_dict)

#     update_cell = DynamicLossScaleUpdateCell(loss_scale_value=2 ** 32, scale_factor=2, scale_window=1000)
#     netwithgrads = BertSquadCell(network, optimizer=optimizer, scale_update_cell=update_cell)
#     model = Model(netwithgrads)
#     callbacks = [TimeMonitor(dataset.get_dataset_size()), LossCallBack(dataset.get_dataset_size()), ckpoint_cb]
#     model.train(epoch_num, dataset, callbacks=callbacks)

In [16]:
from mindspore.train.callback import Callback
from mindspore.train.serialization import load_checkpoint, load_param_into_net
from mindspore.train.model import Model
from mindspore.nn.learning_rate_schedule import LearningRateSchedule, PolynomialDecayLR, WarmUpLR

import collections
import time
class BertLearningRate(LearningRateSchedule):
    """
    Warmup-decay learning rate for Bert network.
    """

    def __init__(self, learning_rate, end_learning_rate, warmup_steps, decay_steps, power):
        super(BertLearningRate, self).__init__()
        self.warmup_flag = False
        if warmup_steps > 0:
            self.warmup_flag = True
            self.warmup_lr = WarmUpLR(learning_rate, warmup_steps)
        self.decay_lr = PolynomialDecayLR(learning_rate, end_learning_rate, decay_steps, power)
        self.warmup_steps = Tensor(np.array([warmup_steps]).astype(np.float32))

        self.greater = mindspore.ops.Greater()
        self.one = Tensor(np.array([1.0]).astype(np.float32))
        self.cast = mindspore.ops.Cast()

    def construct(self, global_step):
        decay_lr = self.decay_lr(global_step)
        if self.warmup_flag:
            is_warmup = self.cast(self.greater(self.warmup_steps, global_step), mstype.float32)
            warmup_lr = self.warmup_lr(global_step)
            lr = (self.one - is_warmup) * decay_lr + is_warmup * warmup_lr
        else:
            lr = decay_lr
        return lr


class LossCallBack(Callback):
    """
    Monitor the loss in training.

    If the loss is NAN or INF terminating training.

    Note:
        If per_print_times is 0 do not print loss.

    Args:
        per_print_times (int): Print loss every times. Default: 1.
    """

    def __init__(self, per_print_times=1, rank_ids=0):
        super(LossCallBack, self).__init__()
        if not isinstance(per_print_times, int) or per_print_times < 0:
            raise ValueError("print_step must be int and >= 0.")
        self._per_print_times = per_print_times
        self.rank_id = rank_ids
        self.time_stamp_first = get_ms_timestamp()

    def step_end(self, run_context):
        """Monitor the loss in training."""
        time_stamp_current = get_ms_timestamp()
        cb_params = run_context.original_args()
        print("time: {}, epoch: {}, step: {}, outputs are {}".format(time_stamp_current - self.time_stamp_first,
                                                                     cb_params.cur_epoch_num,
                                                                     cb_params.cur_step_num,
                                                                     str(cb_params.net_outputs)))
        with open("./loss_{}.log".format(self.rank_id), "a+") as f:
            f.write("time: {}, epoch: {}, step: {}, loss: {}".format(
                time_stamp_current - self.time_stamp_first,
                cb_params.cur_epoch_num,
                cb_params.cur_step_num,
                str(cb_params.net_outputs[0].asnumpy())))
            f.write('\n')


def get_ms_timestamp():
    t = time.time()
    return int(round(t * 1000))

In [17]:
for i in data_set:
    print(len(i))
    print(i)
    break

9
[Tensor(shape=[1, 256], dtype=Int32, value=
[[   0, 3394,   21 ...    1,    1,    1]]), Tensor(shape=[1, 256], dtype=Int32, value=
[[0, 0, 0 ... 0, 0, 0]]), Tensor(shape=[1, 256], dtype=Int32, value=
[[1, 1, 1 ... 0, 0, 0]]), Tensor(shape=[1, 24], dtype=Int32, value=
[[1, 1, 1 ... 0, 0, 0]]), Tensor(shape=[1, 256], dtype=Int32, value=
[[19, 20, -1 ... -1, -1, -1]]), Tensor(shape=[1, 24], dtype=Int32, value=
[[0, 0, 0 ... 0, 0, 0]]), Tensor(shape=[1, 24], dtype=Int32, value=
[[1, 1, 1 ... 0, 0, 0]]), Tensor(shape=[1, 1], dtype=Int32, value=
[[11]]), Tensor(shape=[1, 1], dtype=Int32, value=
[[20]])]


In [18]:
epoch=1

luke_config = BertConfig()
LUKEModel = LukeForReadingComprehension(luke_config)
param_dict = load_checkpoint('./luke-large-qa.ckpt')
load_param_into_net(LUKEModel,param_dict)

# update_cell = DynamicLossScaleUpdateCell(loss_scale_value=2 ** 32, scale_factor=2, scale_window=1000)

lukesquad = LukeForReadingComprehensionWithLoss(LUKEModel)
# lr_schedule = BertLearningRate()
params = lukesquad.trainable_params()
optimizer = mindspore.nn.AdamWeightDecay(params,learning_rate=15e-6, beta1=0.9,beta2=0.98,eps=1e-06)

# lr_schedule
warmup_steps=877
num_train_steps=14629

netwithgrads = LukeSquadCell(lukesquad,optimizer=optimizer)
model = Model(netwithgrads)

loss_monitor = LossCallBack()
model.train(epoch,data_set,callbacks=[loss_monitor],dataset_sink_mode=False)




start_logits: [[-6.3769565 -3.9288216 -3.628428  -6.2168913 -5.2037086 -5.5002394
  -6.2145014 -6.1283865 -5.038344  -6.7068443 -7.0521545 -5.16878
  -5.0806108 -4.1845584 -5.9642477 -4.3139057 -5.461003  -5.9538126
  -5.2949357 -4.5619574 -5.6521254 -5.9900346 -5.14909   -5.3524494
  -6.1287527 -6.1991053 -6.6728706 -5.653833  -4.3856635 -5.195274
  -5.566453  -4.448108  -5.3637195 -4.5569897 -4.5009294 -5.3884645
  -5.252846  -5.12416   -5.934238  -7.3044024 -5.8630257 -4.141479
  -3.802648  -6.5765457 -6.5103106 -4.827878  -6.3929296 -5.0587106
  -6.1132827 -4.2978306 -4.590998  -6.8691077 -5.2707534 -6.3504205
  -5.4268403 -4.8397007 -6.023901  -6.3806944 -6.203143  -4.8382115
  -4.8696356 -5.065989  -4.856864  -5.413134  -5.2270517 -5.342957
  -5.2734103 -4.783379  -4.850018  -5.073468  -4.4455795 -5.778721
  -6.0767236 -5.652123  -5.1519938 -5.0552845 -4.08731   -3.3771334
  -5.836706  -4.4242253 -5.736298  -4.915945  -4.9827657 -6.7050705
  -4.8038406 -5.7566566 -4.465817  -4.93

[ERROR] KERNEL(12621,7f93058f4740,python):2021-10-11-12:24:22.259.320 [mindspore/ccsrc/backend/kernel_compiler/gpu/nn/sparse_softmax_cross_entropy_with_logits_gpu_kernel.h:176] CheckShapeValidation] Labels shape length should be equal to Logits shape length minus 1 for SparseSoftmaxCrossEntropyWithLogits, but got Labels shape length:2, Logits shape length:2


RuntimeError: mindspore/ccsrc/backend/kernel_compiler/gpu/nn/sparse_softmax_cross_entropy_with_logits_gpu_kernel.h:176 CheckShapeValidation] Labels shape length should be equal to Logits shape length minus 1 for SparseSoftmaxCrossEntropyWithLogits, but got Labels shape length:2, Logits shape length:2

# 